In [ ]:
import pandas as pd
import os  # For file path handling
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_columns', 200)  # Set maximum number of columns to display
pd.set_option('display.width', 120) # Set display width for better readability


In [ ]:
# load data
file = "pakistan_weather_2000_2024.csv"
df = pd.read_csv(file, parse_dates=["date"], dayfirst=True, low_memory=False)

# Display the first few rows of the DataFrame
print("Rows,cols:", df.shape)
display(df.head())

#### Understanding the Dataset: Structure, Data Types & Missing Values

Before analyzing or visualizing, it’s important to **understand the structure** of the dataset.

- `df.info()` shows the number of rows, data types of each column, and whether any missing values exist.  
- `df.describe()` gives summary statistics (like min, max, mean) for both numeric and categorical columns.  
- `df.isnull().sum()` counts missing values in each column.

**Why this matters:**  
This step helps identify:
1. Columns that need conversion (e.g., numbers mistakenly stored as text).  
2. Columns with missing data that we may need to fill or drop.  
3. Whether the dataset size and time coverage match our expectations (daily records from 2020–2024).  


In [ ]:
#types and missing values
display(df.info())  #column types

display(df.describe(include="all").T)
 #stats
display(df.isnull().sum().sort_values(ascending=False).head(20))

#### Data Cleaning: Converting Columns to Correct Types

In raw datasets, numeric columns (like `tavg`, `prcp`, or `humidity`) sometimes contain errors (e.g., text values) that prevent proper analysis.

- **force conversion** of numeric columns to the correct type (`float` or `int`).  
- Any invalid values automatically become `NaN` (missing), which will be handled later.  
- The `date` column is explicitly converted to a `datetime` object and sorted.  

 **Why this matters:**  
- Ensures calculations (like averages, sums, correlations) work correctly.  
- Puts all records in chronological order, making time-series analysis possible.  
- Prepares the dataset for **feature engineering** and plotting.  


In [ ]:
# Cell 4: coerce numeric columns (adjust list if you have different names)
num_cols = ['latitude','longitude','elevation','tmin','tmax','tavg','prcp',
            'wspd','humidity','pressure','dew_point','cloud_cover','visibility','temp_range','rainfall_intensity']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

# ensure date is datetime and set index (optional)
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.sort_values('date').reset_index(drop=True)

# quick recheck
display(df[num_cols].info())
display(df[num_cols].isnull().sum().sort_values(ascending=False).head(10))


#### Step 5 — Feature Engineering: Adding Useful Date Features

The dataset has a `date` column, but additional **time-based features** can be extracted:
- `year` → to group by year.  
- `month` → to study seasonal/monthly patterns.  
- `day` → for daily granularity.  
- `dayofweek` → to see if weekdays vs weekends matter.  
- `is_weekend` → boolean flag for Saturday & Sunday.  

**Why this matters:**  
These derived features will help us:
- Compare weather **seasonally** (e.g., hottest month across years).  
- Study trends over **years** (e.g., warming or rainfall changes).  
- Identify differences between **weekdays vs weekends** (useful for human impact studies, air quality, etc.).  

This step is called **feature engineering** and is the foundation of most data science projects.  


In [ ]:
# Cell 5: datetime features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['dayofweek'] = df['date'].dt.dayofweek  # 0=Mon
df['is_weekend'] = df['dayofweek'].isin([5,6])
display(df[['date','year','month','day','dayofweek','is_weekend']].head())

#### Exploratory Data Analysis (EDA)

Now that the dataset is cleaned and enriched with new features, the next step is **Exploratory Data Analysis (EDA)**.

EDA helps us:
- Understand overall trends in the data.
- Detect seasonality (e.g., hottest/coolest months, rainfall cycles).
- Compare patterns across cities and regions.
- Identify unusual data points (outliers).

**summary statistics** and **visualizations** will be used to uncover patterns.


#### Visualization A — Daily Temperature Trend

Plotting the **average daily temperature (`tavg`)** over time.

- A **rolling average (7-day)** smooths short-term noise and highlights seasonal cycles.
- This allows us to see yearly trends (summer vs winter) and whether there are long-term changes from 2020 to 2024.

**What to look for:**
- Do we see clear summer peaks and winter dips each year?
- Any unusual spikes/drops that may indicate anomalies or extreme events?


In [ ]:
# Average temperature over time (single line)
plt.figure(figsize=(12,4))
df.set_index('date')['tavg'].rolling(7, min_periods=1).mean().plot()
plt.title('7-day rolling average of tavg (all data)')
plt.xlabel('Date')
plt.ylabel('tavg')
plt.grid(True)
plt.tight_layout()
plt.show()

#### Visualization B — Monthly Average Temperature (Seasonality)

By grouping all years together, we calculate the **average temperature per month**.

- This shows the **climatological cycle** of Pakistan.
- It helps identify the hottest months (e.g., May–June) and coolest months (e.g., January).

**Why it is important:**
Seasonal analysis is crucial for climate studies, agriculture, and energy planning.  
It also validates whether the dataset matches expected climate patterns.


In [ ]:
# Monthly average tavg (aggregated across all years)
monthly = df.groupby('month')['tavg'].mean()
plt.figure(figsize=(8,4))
monthly.plot(marker='o')
plt.title('Average monthly tavg (2020-2024)')
plt.xlabel('Month (1=Jan)')
plt.ylabel('Mean tavg')
plt.xticks(range(1,13))
plt.grid(True)
plt.tight_layout()
plt.show()


### Visualization C — Comparing Temperature Distributions by City

Using boxplots of `tavg` for different cities:

- Each box shows the **median, spread, and outliers** of daily average temperatures.
- Cities with higher elevation (e.g., northern areas) should have cooler distributions.
- Cities in the plains or coastal regions may show hotter, more humid profiles.

**Importance:**
This allows us to compare **regional climate variations** across Pakistan.


In [ ]:
# Boxplot of tavg by city (only cities with enough samples)
city_counts = df['city'].value_counts()
good_cities = city_counts[city_counts > 200].index.tolist()  # adjust threshold
subset = df[df['city'].isin(good_cities)]

plt.figure(figsize=(12,6))
subset.boxplot(column='tavg', by='city', rot=45)
plt.title('tavg distribution by city')
plt.suptitle('')
plt.ylabel('tavg')
plt.tight_layout()
plt.show()


#### Visualization D — Rainfall Heatmap (Month vs Year)

Aggregate rainfall (`prcp`) by **month and year** and display as a heatmap.

- Darker colors = more rainfall.
- Monsoon months (July–September) are expected to have the highest rainfall totals.

**What to learn:**
- Which years had stronger/weaker monsoons?
- Are there shifts in rainfall timing or intensity across the years?


In [ ]:
# Rainfall heatmap (sum of prcp)
pivot = df.pivot_table(index='month', columns='year', values='prcp', aggfunc='sum')
plt.figure(figsize=(8,5))
plt.imshow(pivot, aspect='auto', origin='lower')
plt.colorbar(label='Total prcp')
plt.yticks(np.arange(0,12), np.arange(1,13))
plt.xticks(np.arange(len(pivot.columns)), pivot.columns)
plt.title('Monthly total rainfall (month vs year)')
plt.xlabel('Year')
plt.ylabel('Month')
plt.tight_layout()
plt.show()


#### Visualization E — Correlation Between Weather Variables

We calculate correlations between numeric variables (`tavg`, `tmin`, `tmax`, `humidity`, `pressure`, etc.).

- Correlation values range from **-1 (strong negative)** to **+1 (strong positive)**.
- A heatmap helps visualize which variables move together.

**Important:**
- Confirms expected relationships (e.g., `tavg` should correlate strongly with `tmin` and `tmax`).
- Helps identify redundant features (useful for modeling).


In [ ]:
num = df.select_dtypes(include='number')
corr = num.corr()
plt.figure(figsize=(8,6))
plt.imshow(corr, vmin=-1, vmax=1)
plt.colorbar()
plt.xticks(range(len(corr)), corr.columns, rotation=90)
plt.yticks(range(len(corr)), corr.columns)
plt.title('Numeric feature correlation matrix (visual)')
plt.tight_layout()
plt.show()


#### Visualization F — Relationship Between Temperature and Humidity

We create a scatterplot of **temperature (`tavg`) vs humidity**:

- Each point is a daily observation.
- A trend line is added to see the overall direction of the relationship.

**Why it is important:**
- Shows whether higher humidity is associated with cooler or warmer days.
- Useful for understanding comfort levels and extreme weather (e.g., hot + humid conditions).


In [ ]:
# Scatter + linear fit (sample)
sample = df.dropna(subset=['tavg','humidity']).sample(frac=0.5, random_state=1)
x = sample['humidity']
y = sample['tavg']
coef = np.polyfit(x, y, 1)
poly = np.poly1d(coef)

plt.figure(figsize=(6,4))
plt.scatter(x, y, s=10, alpha=0.3)
plt.plot(np.sort(x), poly(np.sort(x)))
plt.xlabel('humidity')
plt.ylabel('tavg')
plt.title('tavg vs humidity (scatter + linear fit)')
plt.tight_layout()
plt.show()


#### Detecting Outliers in Temperature

Outliers are values that are unusually high or low compared to the rest of the data.  
We use the **Interquartile Range (IQR) method**:

- Define the lower bound = Q1 − 1.5 × IQR.  
- Define the upper bound = Q3 + 1.5 × IQR.  
- Any values outside this range are considered potential outliers.

**Why do it:**
- Outliers could represent **extreme weather events** (e.g., heatwaves, cold snaps).
- Or there may be **data errors** (e.g., faulty sensor readings).


In [ ]:
# IQR-based outliers for tavg
q1 = df['tavg'].quantile(0.25)
q3 = df['tavg'].quantile(0.75)
iqr = q3 - q1
lower = q1 - 1.5*iqr
upper = q3 + 1.5*iqr
outliers = df[(df['tavg'] < lower) | (df['tavg'] > upper)]
print("Outlier count (tavg):", outliers.shape[0])
display(outliers.head())


#### Baseline Predictive Modeling

A simple **predictive model**:

- Target variable: **average daily temperature (`tavg`)**.
- Features: `tmin`, `tmax`, `prcp`, `humidity`, `pressure`, `dew_point`, and city/month.
- Model: **Random Forest Regressor** (nonlinear, handles missing values reasonably well).

We train the model on part of the dataset and test it on unseen data.

**Metrics:**
- **MAE (Mean Absolute Error):** average size of prediction errors.
- **RMSE (Root Mean Squared Error):** penalizes larger mistakes more heavily.

This provides a baseline — not the most advanced forecast, but shows that weather data has predictive power.


In [ ]:
# Baseline ML: predict tavg using simple features (city encoded, month, wsdp, prcp...)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Prepare dataset (drop rows with missing target)
df_model = df.dropna(subset=['tavg']).copy()

# Basic features
df_model['month'] = df_model['date'].dt.month
features = ['month','tmin','tmax','prcp','wspd','humidity','pressure','dew_point']
# encode city using one-hot (only cities with enough data)
df_model = pd.get_dummies(df_model, columns=['city'], drop_first=True)

# keep features that exist
features = [f for f in features if f in df_model.columns]
X = df_model[features + [c for c in df_model.columns if c.startswith('city_')]]
y = df_model['tavg'].astype(float)

# train/test split by time (recommended) - here simple random split for baseline
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(X_test)

mae = mean_absolute_error(y_test, pred)
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)

print("MAE:", mae, "RMSE:", rmse)


## Conclusions & Insights

From the analysis we can highlight:

1. **Seasonality:** Temperatures follow a strong annual cycle with clear summer/winter differences.
2. **Rainfall:** Monsoon dominates July–September, with significant year-to-year variation.
3. **Regional differences:** Cities show distinct temperature distributions depending on geography.
4. **Correlations:** Expected relationships confirmed (tmin/tmax vs tavg, humidity inverse relation).
5. **Predictive modeling:** Even a simple model explains much of the variation in daily temperature.
